# Spark 101

All the basics! 

Slides: https://www.canva.com/design/DAFnPDJFUyk/ip_ybCdF6yPvjkWfE9tRqw/edit?utm_content=DAFnPDJFUyk&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

## Create Spark Session

In [1]:
#import spark for python! 
import pyspark

#create the spark session
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/30 10:37:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/30 10:37:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Dataframe Basics
 - create dataframe: `spark.createdataframe`
 - see the results: `.show`
 - look at your data: `.describe`, `.dtypes`, `printSchema`
 - select & create columns: `.select`, `col`, `expr`

### create dataframe from a pandas dataframe

In [2]:
import pandas as pd
import numpy as np

In [5]:
np.random.seed(452)

pandas_dataframe = pd.DataFrame(
    dict(n=np.arange(20), group=np.random.choice(list("abc"), 20))
)

pandas_dataframe.head()

,n,group
0,0,c
1,1,a
2,2,a
3,3,a
4,4,a


In [7]:
#create spark dataframe 
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[n: bigint, group: string]

### See the results

<div class='alert alert-box alert-info'>
<b>Note:</b> Spark is lazy and won't display info unless you tell it to
</div>

In [8]:
df.show()

+---+-----+
|  n|group|
+---+-----+
|  0|    c|
|  1|    a|
|  2|    a|
|  3|    a|
|  4|    a|
|  5|    a|
|  6|    c|
|  7|    c|
|  8|    a|
|  9|    a|
| 10|    b|
| 11|    a|
| 12|    a|
| 13|    a|
| 14|    a|
| 15|    a|
| 16|    b|
| 17|    a|
| 18|    c|
| 19|    a|
+---+-----+



### create dataframe from the pydataset

In [9]:
from pydataset import data

In [10]:
df = spark.createDataFrame(data('mpg'))
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

### look at your data

In [21]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [23]:
#describe
df.describe().show()

+-------+------------+-----------------+-----------------+------------------+-----------------+----------+---+------------------+-----------------+----+-------+
|summary|manufacturer|            model|            displ|              year|              cyl|     trans|drv|               cty|              hwy|  fl|  class|
+-------+------------+-----------------+-----------------+------------------+-----------------+----------+---+------------------+-----------------+----+-------+
|  count|         234|              234|              234|               234|              234|       234|234|               234|              234| 234|    234|
|   mean|        null|             null|3.471794871794872|            2003.5|5.888888888888889|      null|4.0|16.858974358974358|23.44017094017094|null|   null|
| stddev|        null|             null|1.291959031083935|4.5096463133204585|1.611534484684289|      null|0.0| 4.255945678889394|5.954643441166448|null|   null|
|    min|        audi|      4runne

In [25]:
#describe vertical
df.describe().show(2, vertical=True)

-RECORD 0--------------------------
 summary      | count              
 manufacturer | 234                
 model        | 234                
 displ        | 234                
 year         | 234                
 cyl          | 234                
 trans        | 234                
 drv          | 234                
 cty          | 234                
 hwy          | 234                
 fl           | 234                
 class        | 234                
-RECORD 1--------------------------
 summary      | mean               
 manufacturer | null               
 model        | null               
 displ        | 3.471794871794872  
 year         | 2003.5             
 cyl          | 5.888888888888889  
 trans        | null               
 drv          | 4.0                
 cty          | 16.858974358974358 
 hwy          | 23.44017094017094  
 fl           | null               
 class        | null               
only showing top 2 rows



In [26]:
#printSchema
df.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- displ: double (nullable = true)
 |-- year: long (nullable = true)
 |-- cyl: long (nullable = true)
 |-- trans: string (nullable = true)
 |-- drv: string (nullable = true)
 |-- cty: long (nullable = true)
 |-- hwy: long (nullable = true)
 |-- fl: string (nullable = true)
 |-- class: string (nullable = true)



In [29]:
#dtypes
df.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

In [31]:
#number of rows
df.count()

234

In [33]:
#number of columns
len(df.columns)

11

### select columns

<div class='alert alert-box alert-info'>
<b>Reminder:</b> Spark is still lazy
</div>

In [34]:
df.select('trans')

DataFrame[trans: string]

In [36]:
df.select('trans').show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [37]:
df.select(df.trans).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [39]:
#select two columns
df.select('trans','hwy').show(5)

+----------+---+
|     trans|hwy|
+----------+---+
|  auto(l5)| 29|
|manual(m5)| 29|
|manual(m6)| 31|
|  auto(av)| 30|
|  auto(l5)| 26|
+----------+---+
only showing top 5 rows



#### save our selected columns

<div class='alert alert-box alert-info'>
<b>Note:</b> You can not save a display output
</div>

In [46]:
df_cols = df.select('model','year').show(5)

+-----+----+
|model|year|
+-----+----+
|   a4|1999|
|   a4|1999|
|   a4|2008|
|   a4|2008|
|   a4|1999|
+-----+----+
only showing top 5 rows



In [45]:
# df_cols.show()

In [47]:
type(df_cols)

NoneType

In [51]:
#remove the .show()
df_cols = df.select('model','year')

In [52]:
df_cols.show()

+------------------+----+
|             model|year|
+------------------+----+
|                a4|1999|
|                a4|1999|
|                a4|2008|
|                a4|2008|
|                a4|1999|
|                a4|1999|
|                a4|2008|
|        a4 quattro|1999|
|        a4 quattro|1999|
|        a4 quattro|2008|
|        a4 quattro|2008|
|        a4 quattro|1999|
|        a4 quattro|1999|
|        a4 quattro|2008|
|        a4 quattro|2008|
|        a6 quattro|1999|
|        a6 quattro|2008|
|        a6 quattro|2008|
|c1500 suburban 2wd|2008|
|c1500 suburban 2wd|2008|
+------------------+----+
only showing top 20 rows



### create columns
 - use basic math operators
 - change column names: `alias`

In [53]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### half the highway mileage

In [56]:
df.hwy / 2

Column<'(hwy / 2)'>

In [61]:
df.select(
    df.hwy,
     (df.hwy / 2).alias('half_hwy')
).show(5)

+---+--------+
|hwy|half_hwy|
+---+--------+
| 29|    14.5|
| 29|    14.5|
| 31|    15.5|
| 30|    15.0|
| 26|    13.0|
+---+--------+
only showing top 5 rows



In [62]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



### select & create columns: `col`

In [63]:
from pyspark.sql.functions import col

In [70]:
df.select(
    df.hwy,
    col('hwy'),
    col('hwy') + 1,
    (col('hwy') + col('cty')).alias('hwy_and_cty')

).show(5)

+---+---+---------+-----------+
|hwy|hwy|(hwy + 1)|hwy_and_cty|
+---+---+---------+-----------+
| 29| 29|       30|         47|
| 29| 29|       30|         50|
| 31| 31|       32|         51|
| 30| 30|       31|         51|
| 26| 26|       27|         42|
+---+---+---------+-----------+
only showing top 5 rows



### select & create columns: `expr`

In [71]:
from pyspark.sql.functions import expr

In [76]:
df.select(
    expr('hwy'),
    expr('hwy + 1'),
    expr('hwy as highway')
).show(5)

+---+---------+-------+
|hwy|(hwy + 1)|highway|
+---+---------+-------+
| 29|       30|     29|
| 29|       30|     29|
| 31|       32|     31|
| 30|       31|     30|
| 26|       27|     26|
+---+---------+-------+
only showing top 5 rows



### create column: `withColumn`

this will keep our initial dataframe and add a new column

In [82]:
df.withColumn(
    'hwy_plus_one', #new column name
    col('hwy') + 1 #how the new column is created
).show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+------------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|hwy_plus_one|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+------------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|          30|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|          30|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|          32|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|          31|
|        audi|        a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|          27|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|          27|
|        audi|        a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|          28|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|          27|
|        audi|a4 quat

In [83]:
df.show(10)

+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+----------+-----+----+---+----------+---+---+---+---+-------+
|        audi|        a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|        a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|        a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|        a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|        a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|        a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|        a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|compact|
|        audi|a4 quattro|  2.0|2008|  4|manual(m6)|  4| 20| 28|  p|compact|
+-----------

## Transforming columns

### built-in functions - math

In [84]:
from pyspark.sql.functions import min, max, sum, count, mean, avg

In [93]:
#use min, max, and calculate average highway mileage
df.select(
    min(df.hwy),
    max(df.hwy),
    avg(df.hwy),
    mean(df.hwy),
    count(col('hwy'))
).show(5)

+--------+--------+-----------------+-----------------+----------+
|min(hwy)|max(hwy)|         avg(hwy)|         avg(hwy)|count(hwy)|
+--------+--------+-----------------+-----------------+----------+
|      12|      44|23.44017094017094|23.44017094017094|       234|
+--------+--------+-----------------+-----------------+----------+



### built-in functions - strings
- `concat`: to concatenate strings
- `lit`: creates literal value of character

In [94]:
from pyspark.sql.functions import concat, lit

In [95]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [99]:
#combine manufacturer and model together
df.select(
    concat(df.manufacturer, df.model)
).show(5)

+---------------------------+
|concat(manufacturer, model)|
+---------------------------+
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
|                     audia4|
+---------------------------+
only showing top 5 rows



In [108]:
#combine manufacturer and model together
df.select(
    df.manufacturer + df.model
).show(5)

+----------------------+
|(manufacturer + model)|
+----------------------+
|                  null|
|                  null|
|                  null|
|                  null|
|                  null|
+----------------------+
only showing top 5 rows



In [103]:
df.select(
    concat(df.manufacturer, lit(' '), df.model).alias('make_and_model')
).show(5)

+--------------+
|make_and_model|
+--------------+
|       audi a4|
|       audi a4|
|       audi a4|
|       audi a4|
|       audi a4|
+--------------+
only showing top 5 rows



In [106]:
#combine city and highway together
df.select(
    df.cty,
    df.hwy,
    concat(df.cty, df.hwy)
).show(5)

+---+---+----------------+
|cty|hwy|concat(cty, hwy)|
+---+---+----------------+
| 18| 29|            1829|
| 21| 29|            2129|
| 20| 31|            2031|
| 21| 30|            2130|
| 16| 26|            1626|
+---+---+----------------+
only showing top 5 rows



In [107]:
df.dtypes

[('manufacturer', 'string'),
 ('model', 'string'),
 ('displ', 'double'),
 ('year', 'bigint'),
 ('cyl', 'bigint'),
 ('trans', 'string'),
 ('drv', 'string'),
 ('cty', 'bigint'),
 ('hwy', 'bigint'),
 ('fl', 'string'),
 ('class', 'string')]

### Regex! 

- `regexp_extract`: use regex to extract data
- `regexp_replace`: use regex to replace data

In [109]:
from pyspark.sql.functions import regexp_extract, regexp_replace

In [112]:
textdf = spark.createDataFrame(
    pd.DataFrame(
        {
            "address": [
                "600 Navarro St ste 600, San Antonio, TX 78205",
                "3130 Broadway St, San Antonio, TX 78209",
                "303 Pearl Pkwy, San Antonio, TX 78215",
                "1255 SW Loop 410!!!!, San - Antonio, TX @78227@",
            ]
        }
    )
)

textdf.show(truncate=False)

+-----------------------------------------------+
|address                                        |
+-----------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205  |
|3130 Broadway St, San Antonio, TX 78209        |
|303 Pearl Pkwy, San Antonio, TX 78215          |
|1255 SW Loop 410!!!!, San - Antonio, TX @78227@|
+-----------------------------------------------+



In [128]:
textdf.select(
    'address',
    regexp_extract('address', r'(\d+)', 1).alias('street_no'),
    regexp_extract('address', r'(\d+)\s(\w+)', 2).alias('street_name'),
    regexp_replace('address', r'[^\w\s]', '').alias('clean_address')
).show(truncate=False)

+-----------------------------------------------+---------+-----------+-------------------------------------------+
|address                                        |street_no|street_name|clean_address                              |
+-----------------------------------------------+---------+-----------+-------------------------------------------+
|600 Navarro St ste 600, San Antonio, TX 78205  |600      |Navarro    |600 Navarro St ste 600 San Antonio TX 78205|
|3130 Broadway St, San Antonio, TX 78209        |3130     |Broadway   |3130 Broadway St San Antonio TX 78209      |
|303 Pearl Pkwy, San Antonio, TX 78215          |303      |Pearl      |303 Pearl Pkwy San Antonio TX 78215        |
|1255 SW Loop 410!!!!, San - Antonio, TX @78227@|1255     |SW         |1255 SW Loop 410 San  Antonio TX 78227     |
+-----------------------------------------------+---------+-----------+-------------------------------------------+



### Filter and Where

In [129]:
df = spark.createDataFrame(data('mpg'))
df

DataFrame[manufacturer: string, model: string, displ: double, year: bigint, cyl: bigint, trans: string, drv: string, cty: bigint, hwy: bigint, fl: string, class: string]

In [130]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [138]:
df.filter(df.cyl == 4).where(col('class') == 'midsize').show()

+------------+------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer| model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------+-----+----+---+----------+---+---+---+---+-------+
|   chevrolet|malibu|  2.4|1999|  4|  auto(l4)|  f| 19| 27|  r|midsize|
|   chevrolet|malibu|  2.4|2008|  4|  auto(l4)|  f| 22| 30|  r|midsize|
|     hyundai|sonata|  2.4|1999|  4|  auto(l4)|  f| 18| 26|  r|midsize|
|     hyundai|sonata|  2.4|1999|  4|manual(m5)|  f| 18| 27|  r|midsize|
|     hyundai|sonata|  2.4|2008|  4|  auto(l4)|  f| 21| 30|  r|midsize|
|     hyundai|sonata|  2.4|2008|  4|manual(m5)|  f| 21| 31|  r|midsize|
|      nissan|altima|  2.5|2008|  4|  auto(av)|  f| 23| 31|  r|midsize|
|      nissan|altima|  2.5|2008|  4|manual(m6)|  f| 23| 32|  r|midsize|
|      toyota| camry|  2.2|1999|  4|manual(m5)|  f| 21| 29|  r|midsize|
|      toyota| camry|  2.2|1999|  4|  auto(l4)|  f| 21| 27|  r|midsize|
|      toyota| camry|  2.4|2008|  4|manual(m5)|  f| 21| 31|  r|m

### When and Otherwise

In [139]:
from pyspark.sql.functions import when

In [150]:
df.select(
    df.hwy,
    when(df.hwy > 27, 'good').alias('good_bad_miles'), #values are null when not in the when
    when(df.hwy > 27, 'good').when(df.hwy > 25, 'medium').otherwise('bad').alias('good_bad_miles') #chain whens
).show(10)

+---+--------------+--------------+
|hwy|good_bad_miles|good_bad_miles|
+---+--------------+--------------+
| 29|          good|          good|
| 29|          good|          good|
| 31|          good|          good|
| 30|          good|          good|
| 26|          null|        medium|
| 26|          null|        medium|
| 27|          null|        medium|
| 26|          null|        medium|
| 25|          null|           bad|
| 28|          good|          good|
+---+--------------+--------------+
only showing top 10 rows



### sorting and ordering

In [151]:
df.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [153]:
df.sort(df.cty).show(5)

+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|manufacturer|              model|displ|year|cyl|     trans|drv|cty|hwy| fl| class|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|        jeep| grand cherokee 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|        durango 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|   suv|
|       dodge|  dakota pickup 4wd|  4.7|2008|  8|  auto(l5)|  4|  9| 12|  e|pickup|
|       dodge|ram 1500 pickup 4wd|  4.7|2008|  8|manual(m6)|  4|  9| 12|  e|pickup|
+------------+-------------------+-----+----+---+----------+---+---+---+---+------+
only showing top 5 rows



In [154]:
df.sort(df.cty.desc()).show(5)

+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|manufacturer|     model|displ|year|cyl|     trans|drv|cty|hwy| fl|     class|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
|  volkswagen|new beetle|  1.9|1999|  4|manual(m5)|  f| 35| 44|  d|subcompact|
|  volkswagen|     jetta|  1.9|1999|  4|manual(m5)|  f| 33| 44|  d|   compact|
|  volkswagen|new beetle|  1.9|1999|  4|  auto(l4)|  f| 29| 41|  d|subcompact|
|      toyota|   corolla|  1.8|2008|  4|manual(m5)|  f| 28| 37|  r|   compact|
|       honda|     civic|  1.6|1999|  4|manual(m5)|  f| 28| 33|  r|subcompact|
+------------+----------+-----+----+---+----------+---+---+---+---+----------+
only showing top 5 rows



In [163]:
df.sort(df.cty.desc(), df.year).tail(1)

[Row(manufacturer='jeep', model='grand cherokee 4wd', displ=4.7, year=2008, cyl=8, trans='auto(l5)', drv='4', cty=9, hwy=12, fl='e', class='suv')]

### Grouping and Aggregating

In [164]:
#groupby/groupBy
df.groupBy(df.cyl)

In [166]:
df.groupBy(df.cyl).agg(mean(df.hwy)).show()

+---+-----------------+
|cyl|         avg(hwy)|
+---+-----------------+
|  6|22.82278481012658|
|  8|17.62857142857143|
|  4|28.80246913580247|
|  5|            28.75|
+---+-----------------+



In [167]:
df.groupBy(df.cyl).agg(mean(df.hwy)).sort(df.cyl).show()

+---+-----------------+
|cyl|         avg(hwy)|
+---+-----------------+
|  4|28.80246913580247|
|  5|            28.75|
|  6|22.82278481012658|
|  8|17.62857142857143|
+---+-----------------+



In [168]:
#rollup
df.rollup(df.cyl).agg(mean(df.hwy)).show()

+----+-----------------+
| cyl|         avg(hwy)|
+----+-----------------+
|   4|28.80246913580247|
|null|23.44017094017094|
|   6|22.82278481012658|
|   8|17.62857142857143|
|   5|            28.75|
+----+-----------------+



### Crosstabs and Pivot Tables

In [170]:
#crosstab
df.crosstab('class', 'cyl').show()

+----------+---+---+---+---+
| class_cyl|  4|  5|  6|  8|
+----------+---+---+---+---+
|   midsize| 16|  0| 23|  2|
|subcompact| 21|  2|  7|  5|
|   2seater|  0|  0|  0|  5|
|    pickup|  3|  0| 10| 20|
|   minivan|  1|  0| 10|  0|
|       suv|  8|  0| 16| 38|
|   compact| 32|  2| 13|  0|
+----------+---+---+---+---+



In [171]:
#groupby and pivot
df.groupby('class').pivot('cyl').mean('hwy').show()

+----------+------------------+----+------------------+------------------+
|     class|                 4|   5|                 6|                 8|
+----------+------------------+----+------------------+------------------+
|subcompact| 30.80952380952381|28.5|24.714285714285715|              21.6|
|   compact|          29.46875|29.0|25.307692307692307|              null|
|   minivan|              24.0|null|              22.2|              null|
|       suv|             23.75|null|              18.5|16.789473684210527|
|   midsize|           29.1875|null| 26.26086956521739|              24.0|
|    pickup|20.666666666666668|null|              17.9|              15.8|
|   2seater|              null|null|              null|              24.8|
+----------+------------------+----+------------------+------------------+



### Handling Missing Data

- `.na.fill`: to replace missing values with a specified value
- `.na.drop`: to drop rows containing missing values

In [172]:
df = spark.createDataFrame(
    pd.DataFrame(
        {"x": [1, 2, np.nan, 4, 5, np.nan], "y": [np.nan, 0, 0, 3, 1, np.nan]}
    )
)
df.show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|NaN|0.0|
|4.0|3.0|
|5.0|1.0|
|NaN|NaN|
+---+---+



In [173]:
df.na.drop().show()

+---+---+
|  x|  y|
+---+---+
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [177]:
df.na.drop(subset='x').show()

+---+---+
|  x|  y|
+---+---+
|1.0|NaN|
|2.0|0.0|
|4.0|3.0|
|5.0|1.0|
+---+---+



In [180]:
df.na.fill(100).show()

+-----+-----+
|    x|    y|
+-----+-----+
|  1.0|100.0|
|  2.0|  0.0|
|100.0|  0.0|
|  4.0|  3.0|
|  5.0|  1.0|
|100.0|100.0|
+-----+-----+



In [182]:
df.na.fill(100, subset='y').na.fill(-5, subset='x').show()

+----+-----+
|   x|    y|
+----+-----+
| 1.0|100.0|
| 2.0|  0.0|
|-5.0|  0.0|
| 4.0|  3.0|
| 5.0|  1.0|
|-5.0|100.0|
+----+-----+



### More Dataframe Manipulation Examples

In [183]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
df = spark.createDataFrame(weather)
df.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



#### shape of df

In [184]:
df.count(), len(df.columns)

(1461, 6)

#### start and end date

In [186]:
df.select(
    min(df.date), max(df.date)
).show()

+----------+----------+
| min(date)| max(date)|
+----------+----------+
|2012-01-01|2015-12-31|
+----------+----------+



#### Find the total rainfall per month

In [202]:
from pyspark.sql.functions import month, year, quarter, round

In [209]:
(
    df.withColumn('themonth', month('date'))
    .groupby('themonth')
    .agg(round(sum('precipitation')).alias('total_rain'))
    .sort('themonth')
).show()

+--------+----------+
|themonth|total_rain|
+--------+----------+
|       1|     466.0|
|       2|     422.0|
|       3|     606.0|
|       4|     375.0|
|       5|     208.0|
|       6|     133.0|
|       7|      48.0|
|       8|     164.0|
|       9|     235.0|
|      10|     503.0|
|      11|     643.0|
|      12|     623.0|
+--------+----------+



### Joins

In [210]:
users = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4, 5, 6],
            "name": ["bob", "joe", "sally", "adam", "jane", "mike"],
            "role_id": [1, 2, 3, 3, np.nan, np.nan],
        }
    )
)
roles = spark.createDataFrame(
    pd.DataFrame(
        {
            "id": [1, 2, 3, 4],
            "name": ["admin", "author", "reviewer", "commenter"],
        }
    )
)
print("--- users ---")
users.show()
print("--- roles ---")
roles.show()

--- users ---
+---+-----+-------+
| id| name|role_id|
+---+-----+-------+
|  1|  bob|    1.0|
|  2|  joe|    2.0|
|  3|sally|    3.0|
|  4| adam|    3.0|
|  5| jane|    NaN|
|  6| mike|    NaN|
+---+-----+-------+

--- roles ---
+---+---------+
| id|     name|
+---+---------+
|  1|    admin|
|  2|   author|
|  3| reviewer|
|  4|commenter|
+---+---------+



In [211]:
users.join(roles, on= users.role_id == roles.id).show()

+---+-----+-------+---+--------+
| id| name|role_id| id|    name|
+---+-----+-------+---+--------+
|  1|  bob|    1.0|  1|   admin|
|  2|  joe|    2.0|  2|  author|
|  3|sally|    3.0|  3|reviewer|
|  4| adam|    3.0|  3|reviewer|
+---+-----+-------+---+--------+



In [212]:
users.join(roles, on= users.role_id == roles.id, how='left').show()

+---+-----+-------+----+--------+
| id| name|role_id|  id|    name|
+---+-----+-------+----+--------+
|  1|  bob|    1.0|   1|   admin|
|  2|  joe|    2.0|   2|  author|
|  3|sally|    3.0|   3|reviewer|
|  4| adam|    3.0|   3|reviewer|
|  5| jane|    NaN|null|    null|
|  6| mike|    NaN|null|    null|
+---+-----+-------+----+--------+



In [213]:
users.join(roles, on= users.role_id == roles.id, how='right').show()

+----+-----+-------+---+---------+
|  id| name|role_id| id|     name|
+----+-----+-------+---+---------+
|   1|  bob|    1.0|  1|    admin|
|   2|  joe|    2.0|  2|   author|
|   4| adam|    3.0|  3| reviewer|
|   3|sally|    3.0|  3| reviewer|
|null| null|   null|  4|commenter|
+----+-----+-------+---+---------+



In [216]:
users.toPandas()

,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN
